In [1]:
%config Completer.use_jedi = False

별점 데이터이긴 하나, 시청횟수(implicit)데이터로 해석해서 보자   
그리고 3점 미만의 데이터는 선호하지 않는다고 가정

In [2]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [5]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# 타임스탬프는 제거
out_timestamp = ['user_id', 'count', 'movie_id']
ratings = ratings[out_timestamp]
ratings.head()

,user_id,count,movie_id
0,1,5,1193
1,1,3,661
2,1,3,914
3,1,4,3408
4,1,5,2355


# EDA
- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개 (인기순)

In [8]:
# 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [9]:
# 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [10]:
# 영화 이름이랑 같이 보기 위해 merge
data = pd.merge(movies, ratings, left_on='movie_id', right_on='movie_id', how='inner')
data.tail()

,movie_id,title,genre,user_id,count
836473,3952,"Contender, The (2000)",Drama|Thriller,5682,3
836474,3952,"Contender, The (2000)",Drama|Thriller,5812,4
836475,3952,"Contender, The (2000)",Drama|Thriller,5831,3
836476,3952,"Contender, The (2000)",Drama|Thriller,5837,4
836477,3952,"Contender, The (2000)",Drama|Thriller,5998,4


In [11]:
# 인기 많은 영화
movie_count = data.groupby('title')['count'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

# 선호하는 영화 5가지 추가

In [12]:
my_favorite = ['Jurassic Park (1993)',
               'Saving Private Ryan (1998)',
               '101 Dalmatians (1961)',
               'Aladdin (1992)',
               'Hercules (1997)']
my_favorite_id = movies[movies['title'].isin(my_favorite)]
my_favorite_id

,movie_id,title,genre
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
584,588,Aladdin (1992),Animation|Children's|Comedy|Musical
1526,1566,Hercules (1997),Adventure|Animation|Children's|Comedy|Musical
1959,2028,Saving Private Ryan (1998),Action|Drama|War
2016,2085,101 Dalmatians (1961),Animation|Children's


In [13]:
# 'chromis'이라는 user_id가 위의 영화를 5번씩 봄
my_movielist = pd.DataFrame({'user_id': ['chromis']*5, 'count':['5']*5, 'movie_id': my_favorite_id['movie_id']})

if not ratings.isin({'user_id':['chromis']})['user_id'].any():  # user_id에 my_id 데이터가 없다면
    ratings = ratings.append(my_movielist)                    # 임의로 만든 my_favorite_id 데이터를 추가          

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,count,movie_id
1000203,6040,3,1090
1000205,6040,5,1094
1000206,6040,5,562
1000207,6040,4,1096
1000208,6040,4,1097
476,chromis,5,480
584,chromis,5,588
1526,chromis,5,1566
1959,chromis,5,2028
2016,chromis,5,2085


In [14]:
data = pd.merge(movies, ratings, left_on='movie_id', right_on='movie_id', how='inner')
data.tail()

,movie_id,title,genre,user_id,count
836478,3952,"Contender, The (2000)",Drama|Thriller,5682,3
836479,3952,"Contender, The (2000)",Drama|Thriller,5812,4
836480,3952,"Contender, The (2000)",Drama|Thriller,5831,3
836481,3952,"Contender, The (2000)",Drama|Thriller,5837,4
836482,3952,"Contender, The (2000)",Drama|Thriller,5998,4


# 인덱싱

In [15]:
# 고유한 유저, 제목 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# 추천 시스템, 자연어처리에서 주로 indexing을 함
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [16]:
print(user_to_idx['chromis'])
print(movie_to_idx['Jurassic Park (1993)'])

5838
462


In [17]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 

temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

data.tail()

user_id column indexing OK!!
title column indexing OK!!


,movie_id,title,genre,user_id,count
836478,3952,3627,Drama|Thriller,2696,3
836479,3952,3627,Drama|Thriller,2161,4
836480,3952,3627,Drama|Thriller,1931,3
836481,3952,3627,Drama|Thriller,2162,4
836482,3952,3627,Drama|Thriller,3134,4


# CSR matrix 생성

In [18]:
data['count'] = data['count'].astype(int)
data['count']

0         5
1         4
2         4
3         5
4         5
         ..
836478    3
836479    4
836480    3
836481    4
836482    4
Name: count, Length: 836483, dtype: int64

In [20]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data['count'], (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [22]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [23]:
# Implicit AlternatingLeastSquares 모델의 선언

# 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
# 2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
# 3. use_gpu : GPU를 사용할 것인지 
# 4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지
# 1, 4를 늘릴 수록 오버피팅 주의
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [24]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [25]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [27]:
# my_id의 벡터
my_id_ = user_to_idx['chromis']
my_vector = als_model.user_factors[my_id_]
my_vector

array([-0.38554427,  0.23215708,  0.22222392,  0.31532073, -1.2417275 ,
       -0.4132351 , -0.22878218,  0.0855615 , -0.18601291, -0.01471598,
       -0.48111108, -0.54638934,  0.31813133,  1.2933977 , -0.20486   ,
       -0.2132461 ,  0.7716071 , -0.1723135 ,  0.6595237 ,  1.3764448 ,
        0.25930053,  1.0305026 ,  0.06800374,  0.20303118, -0.11016093,
        0.2325423 , -0.5669005 , -0.99879354,  0.18326314, -0.34553272,
        0.65511554, -0.07094454,  1.0092012 ,  0.18971851,  0.5050073 ,
       -0.91428447,  0.53672427, -0.9869515 ,  0.4787666 , -0.418256  ,
        0.69228584,  0.16964582, -0.08711607, -0.1716071 , -0.11696184,
        0.06425721,  0.42261055,  0.9128025 , -0.51443416,  0.15295026,
        0.01414427,  0.06851576, -0.12416703, -0.14141677, -0.5530659 ,
       -0.34895664,  0.11389389, -0.18535046, -0.2979741 , -0.6266275 ,
        0.31949678, -0.14378579,  0.15431154, -0.22106764, -0.616288  ,
        0.43170825, -0.4038796 ,  0.07350392,  0.08785509, -0.05

In [29]:
# ToyStory의 벡터
jurassic_idx = movie_to_idx['Jurassic Park (1993)']
jurassic_vector = als_model.item_factors[jurassic_idx]
jurassic_vector

array([ 0.02204767, -0.01709229,  0.0222594 ,  0.00919449, -0.03373662,
       -0.00523083, -0.00051435,  0.02128402, -0.01521886,  0.01166473,
       -0.00076851,  0.00315026,  0.00335883,  0.03803603,  0.00739363,
        0.00785332,  0.02590881,  0.00628625,  0.03872775,  0.00919532,
        0.02161596,  0.05966716, -0.01478059,  0.02095484, -0.00226008,
        0.01491353,  0.00037897,  0.00607496,  0.01373228,  0.00961335,
        0.01775609,  0.00510369,  0.02457232,  0.01278973, -0.00627568,
       -0.00612714,  0.01236975, -0.04537435,  0.01447429,  0.0365887 ,
       -0.01493203,  0.02212249,  0.01771977, -0.00753564,  0.03129242,
        0.01010097,  0.00481004,  0.02929473,  0.00564855, -0.00180872,
        0.00514125,  0.00806041,  0.00405615,  0.01386695, -0.02982303,
       -0.03630549,  0.02444779,  0.00566382, -0.01150832, -0.01270406,
       -0.01336376,  0.00423846, -0.00464143,  0.00415029, -0.01871725,
        0.03256608, -0.00147753,  0.01101692,  0.00403876,  0.01

In [30]:
# chromis과 Jurassic Park를 내적하는 코드
np.dot(my_vector, jurassic_vector)

# factor와 iteration을 늘리면 높일 수 있음
# 이마저도 validation이 되어있지는 않음

0.4883075

In [31]:
# 라이언 일병 구하기는 선호할까?

ryan_idx = movie_to_idx['Saving Private Ryan (1998)']
ryan_vector = als_model.item_factors[ryan_idx]
ryan_vector

array([-0.00863961,  0.01757316,  0.00540848,  0.01090585, -0.04302669,
        0.00290677,  0.00760246,  0.01193882, -0.00997776,  0.00394402,
       -0.01000271,  0.02714979,  0.02415839,  0.01667426,  0.0081657 ,
       -0.01384252,  0.03565389,  0.01724183,  0.015243  ,  0.03394166,
        0.03453815,  0.04373143, -0.02371888,  0.01121053,  0.01277383,
       -0.02459559, -0.01346487, -0.01783264,  0.01918596,  0.00742108,
        0.01605694,  0.01495344,  0.03917889,  0.02529074,  0.05790202,
       -0.02247073,  0.00485395, -0.02647264,  0.0043178 , -0.03992187,
       -0.01161251,  0.00296851,  0.00425027,  0.01732757, -0.01134896,
        0.01398673,  0.01355338,  0.0001067 ,  0.02795647,  0.02067286,
        0.03069018,  0.00076459,  0.0018956 ,  0.00472309,  0.02109858,
       -0.01374349,  0.0380499 , -0.02827894, -0.00244432, -0.05208305,
        0.01752842,  0.01757613,  0.01016309, -0.01178655, -0.02391046,
        0.00404496,  0.00621083,  0.01946852,  0.01221969, -0.02

In [33]:
np.dot(my_vector, ryan_vector)

0.5533898

쥬라기 공룡보다 더 선호한다고 한다  
실제로 그렇긴 한데... 애니메이션이랑 비교하면?

In [34]:
hercules_idx = movie_to_idx['Hercules (1997)']
hercules_vector = als_model.item_factors[hercules_idx]
hercules_vector

array([ 0.00746138,  0.02275591,  0.00743458,  0.00525347,  0.00651651,
        0.00810904,  0.01513133,  0.00185762, -0.00290646,  0.01126913,
       -0.01689399, -0.02583956,  0.01859002,  0.02697263, -0.00318082,
       -0.0075245 ,  0.01452739,  0.00942905,  0.00820493,  0.01354403,
       -0.00340311,  0.0135978 ,  0.01094662,  0.01024856,  0.00012229,
        0.01153092, -0.00966936, -0.000561  , -0.01558557, -0.01789964,
        0.0044241 ,  0.00142582,  0.03578516,  0.02125624,  0.0034116 ,
       -0.00058728,  0.00834122,  0.0013093 ,  0.01699295,  0.00825944,
        0.01559609,  0.00915755,  0.00427909,  0.00088425, -0.01544033,
        0.00801624,  0.01340888,  0.03574264, -0.00119224,  0.00978482,
       -0.00364216,  0.01306215,  0.00217746, -0.00807039, -0.01611638,
        0.02707908, -0.00937473,  0.02231801,  0.00810137,  0.009166  ,
        0.01844274,  0.00416526,  0.00174702,  0.00022288,  0.01949962,
        0.00713353, -0.01110583,  0.01617453,  0.00964191,  0.01

In [35]:
np.dot(my_vector, hercules_vector)

0.33808562

역시 애니메이션에서는 살짝 낮은 편

# 비슷한 영화 찾기
실제로도 라이언 일병 구하기를 가장 좋아하긴 한데   
그렇다면 이와 비슷한 영화는?

In [36]:
favorite_movie = 'Saving Private Ryan (1998)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(1810, 0.99999994),
 (509, 0.69929165),
 (106, 0.6584161),
 (305, 0.5150397),
 (1116, 0.45133075),
 (1532, 0.4220056),
 (439, 0.4130607),
 (2629, 0.40968493),
 (2325, 0.36027172),
 (573, 0.35845795),
 (1124, 0.35622546),
 (2600, 0.34615228),
 (35, 0.32452023),
 (2186, 0.32445893),
 (3284, 0.3159247)]

In [37]:
# movie_to_idx 를 뒤집어, index로부터 movie의 제목을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

def get_similar_movie(movie_title: str):
    title_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(title_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

get_similar_movie('Saving Private Ryan (1998)')

['Saving Private Ryan (1998)',
 "Schindler's List (1993)",
 'Braveheart (1995)',
 'Shawshank Redemption, The (1994)',
 'Boat, The (Das Boot) (1981)',
 'Good Will Hunting (1997)',
 'Fugitive, The (1993)',
 'Simon Sez (1999)',
 'Matrix, The (1999)',
 'Silence of the Lambs, The (1991)']

쉰들러리스트, 매트릭스, 양들의 침묵 등... 어느정도 만족스럽다

In [39]:
get_similar_movie('Hercules (1997)')

['Hercules (1997)',
 'Hunchback of Notre Dame, The (1996)',
 'Anastasia (1997)',
 'Tarzan (1999)',
 'Pocahontas (1995)',
 'Mulan (1998)',
 'Aladdin and the King of Thieves (1996)',
 'Rescuers Down Under, The (1990)',
 'Return of Jafar, The (1993)',
 'Quest for Camelot (1998)']

디즈니 영화엔 디즈니 영화들이 나온다

In [40]:
get_similar_movie('Jurassic Park (1993)')

['Jurassic Park (1993)',
 'Men in Black (1997)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Matrix, The (1999)',
 'Lost World: Jurassic Park, The (1997)',
 'Schlafes Bruder (Brother of Sleep) (1995)',
 'Independence Day (ID4) (1996)',
 'Braveheart (1995)',
 'Fifth Element, The (1997)']

맨 인 블랙, 토탈리콜, 터미네이터2, 인디펜던스 데이 등 SF영화들이 나온다(싱기)

# 영화 추천하기

In [41]:
user = user_to_idx['chromis']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(575, 0.43730804),
 (350, 0.41423285),
 (106, 0.39797762),
 (1419, 0.3675902),
 (2434, 0.3514824),
 (1689, 0.3477141),
 (509, 0.34502336),
 (2600, 0.30940875),
 (569, 0.26283956),
 (1861, 0.25551358),
 (725, 0.24987848),
 (0, 0.2399936),
 (945, 0.22954535),
 (722, 0.2269634),
 (2114, 0.22208343),
 (1520, 0.22005314),
 (2153, 0.21998253),
 (2056, 0.21363541),
 (2845, 0.21222278),
 (47, 0.20584123)]

In [42]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Beauty and the Beast (1991)',
 'Lion King, The (1994)',
 'Braveheart (1995)',
 'Men in Black (1997)',
 'Tarzan (1999)',
 'Mulan (1998)',
 "Schindler's List (1993)",
 'American Beauty (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Little Mermaid, The (1989)',
 'Hunchback of Notre Dame, The (1996)',
 'Toy Story (1995)',
 'Alice in Wonderland (1951)',
 'Independence Day (ID4) (1996)',
 "Bug's Life, A (1998)",
 'Anastasia (1997)',
 'Prince of Egypt, The (1998)',
 'Antz (1998)',
 'Toy Story 2 (1999)',
 'Pocahontas (1995)']

디즈니 영화 2개를 넣었더니 대부분 디즈니 영화를 추천하는 편이다   
그 중에서도 쉰들러 리스트, 인디펜던스 데이, 맨인 블랙은 그럼에도 추천되는 편!

In [45]:
man = movie_to_idx['Men in Black (1997)']
explain = als_model.explain(user, csr_data, itemid=man)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]
# 맨 인 블랙를 추천한 기여도 (합한 것이 위에 나타난 점수)

[('Jurassic Park (1993)', 0.33159741251118),
 ('Aladdin (1992)', 0.013313768478360705),
 ('Saving Private Ryan (1998)', 0.011035066802662105),
 ('Hercules (1997)', 0.006560929248583447),
 ('101 Dalmatians (1961)', 0.0007416080957464948)]

기여도를 보니 쥬라기 공원이 대부분 기여한 편

In [47]:
poca = movie_to_idx['Pocahontas (1995)']
explain = als_model.explain(user, csr_data, itemid=poca)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Hercules (1997)', 0.10016446888186656),
 ('Aladdin (1992)', 0.0565582859790658),
 ('101 Dalmatians (1961)', 0.02872234904213944),
 ('Jurassic Park (1993)', 0.016082925155113427),
 ('Saving Private Ryan (1998)', 0.0019630027589435106)]

포카혼타스는 애니메이션인 덕분에 헤라클래스가 대부분을 차지, 알라딘도 도움